In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 9장. 전이학습 실습

### ①  VGG16 모델 확인 → Cats Vs Dogs datasets 로드  → 이미지 확인 , 크기 조정 및 회전


In [ ]:
#  기본 모듈 임포트
import numpy as np
import tensorflow as tf
from tensorflow import keras

#  Imagenet 을 학습한 VGG16 모듈 불러오기 
test_model = keras.applications.VGG16(
    weights="imagenet",  # 이미지넷을 학습한 가중치 로드
    input_tensor  =  None , 
    input_shape  =  None, 
    include_top=True,  # classifer 포함
)

test_model.summary()


In [ ]:
#   Cats Vs Dogs datasets 로드
import tensorflow_datasets as tfds
tfds.disable_progress_bar()  #데이터 로드시 진행표시 제거
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"], 
    as_supervised=True,)

print("Number of training samples: %d" %tf.data.experimental.cardinality(train_ds))
print("Number of validation samples: %d" % tf.data.experimental.cardinality(validation_ds))
print("Number of test samples: %d" % tf.data.experimental.cardinality(test_ds))


In [ ]:
#   dataset 이미지 확인
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i, (image, label) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(image)
    plt.title(int(label))
    plt.axis("off")


In [ ]:
#  이미지 사이즈를 150,150으로 조정
size = (150, 150)
train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

#  batch size 조정
batch_size = 32
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=10)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=10)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=10)


In [ ]:
# 이미지 데이터 augmentation
# 데이터셋이 작을경우 이미지에 다양한 변형(회전, 상하좌우 뒤집기)을 줘, 학습모델의 성능을 향상
from tensorflow import keras
from tensorflow.keras import layers
data_augmentation = keras.Sequential([
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ])

#이미지 확인
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10,10))
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image,0), training=True)
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(int(labels[i]))
        plt.axis("off")
        

In [ ]:
#  VGG16 모델 로드 (classifier 제외)
base_model = keras.applications.VGG16(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
)

# 파라미터  학습이 안되도록 설정
base_model.trainable = False  

base_model.summary()


In [ ]:
#  VGG16 모델 로드 (classifier 제외)
inputs = keras.Input(shape=(150,150,3))
x = data_augmentation(inputs) # Apply random data augmentation
x = keras.layers.BatchNormalization()(x) #배치 정규화 layer 추가
x = base_model(x, training=False)
x = keras.layers.Flatten()(x) # classifier layer 추가
x = keras.layers.Dense(500)(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
model.summary()

#  모델 컴파일
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

#  Cats Vs Dogs datasets 학습
epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)


In [ ]:
!pwd